In [80]:
# Imports
from buildingmotif import BuildingMOTIF
from rdflib import Namespace, Literal
from buildingmotif.dataclasses import Model
from buildingmotif.dataclasses import Library
from buildingmotif.namespaces import BRICK, REF
import csv

2023-01-20 10:57:28,343 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject
2023-01-20 10:57:28,343 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice


In [150]:
# getting libraries and templates
brick = Library.load(ontology_graph="../BuildingMOTIF/libraries/brick/Brick-full.ttl")
lib = Library.load(directory="/Users/lazlopaul/Desktop/VolttronSemantics/vltt-templates")
topic_name = lib.get_template_by_name("volttron-topic")
point_list = lib.get_template_by_name("volttron-point-list")
device_config = lib.get_template_by_name("volttron-device-config")

2023-01-20 11:56:07,881 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject
2023-01-20 11:56:07,881 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice


In [157]:
# Binding namespaces and initializing model
VLTT = Namespace('https://www.doesntexist.org/volttronsemantics#')
BLDG = Namespace('urn:bldg/')
bm = BuildingMOTIF("sqlite://") # in-memory instance
model = Model.create(BLDG, description="Test Model") 
model.graph.bind("VLTT", VLTT)
model.graph.bind("ref", REF)

In [158]:
#read csv files and create list of dicts
def read_csv(path):
    line = 0
    contents = []
    with open(path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            contents.append(row)
        return contents

# get templates using list of dictionaries from read_csv
def get_templates(dct_list, key):
    templates = {}
    type_set = {item[key] for item in dct_list}
    for type in type_set:
        templates[type] = brick.get_template_by_name(BRICK[type])
    return templates

# add points given an item dictionary from the entity table
def add_points(item, directory = 'files/volttron_configs'):
    filepath = directory + '/' + item.get('point_list')
    points = read_csv(filepath)
    templates = get_templates(points, 'Brick Type')
    name = item.get('URI')

    for point in points:
        type = point.get('Brick Type')
        point_name = name + '.' + point.get('Point Name')
        # print(point_name)
        model.add_graph(
            templates[type].evaluate(
                {
                    "name":BLDG[point_name]
                }
            )
        )
        model.graph.add((BLDG[item['URI']], BRICK.hasPoint, BLDG[point_name]))


In [159]:
filename  = '/Users/lazlopaul/Desktop/VolttronSemantics/files/entity_table.csv'
contents = read_csv(filename)
templates = get_templates(contents, 'type')

In [160]:
#Instantiate templates and add to model
for item in contents:
    type = item.get('type')
    name = item.get('URI')
    # print(name)

    model.add_graph(
        templates[type].evaluate(
            {
                "name":BLDG[name]
            }
        )
    )

    if item.get('point_list', None):
        model.add_graph(
            point_list.evaluate(
                {
                    "name":BLDG[name+".point_list"],
                    "config-path": Literal(item.get('point_list'))
                }
            )
        )
        model.graph.add((BLDG[name], REF['hasExternalReference'], BLDG[name+".point_list"]))

        add_points(item)

    if item.get('config', None):
        model.add_graph(
            device_config.evaluate(
                {
                    "name":BLDG[name+".config"],
                    "config-path": Literal(item.get('config'))
                }
            )
        )
        model.graph.add((BLDG[name], REF['hasExternalReference'], BLDG[name+".config"]))

In [161]:
print(model.graph.serialize())

@prefix VLTT: <https://www.doesntexist.org/volttronsemantics#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/AdminSpace> a brick:Room .

<urn:bldg/Bldg> a brick:Building .

<urn:bldg/Boiler1> a brick:Boiler .

<urn:bldg/Boiler2> a brick:Boiler .

<urn:bldg/FCU1> a brick:FCU ;
    brick:hasPoint <urn:bldg/FCU1.fanLevel>,
        <urn:bldg/FCU1.mode>,
        <urn:bldg/FCU1.on>,
        <urn:bldg/FCU1.targetTemperature>,
        <urn:bldg/FCU1.temperature>,
        <urn:bldg/FCU1.time> ;
    ref:hasExternalReference <urn:bldg/FCU1.config>,
        <urn:bldg/FCU1.point_list> .

<urn:bldg/FCU2> a brick:FCU ;
    brick:hasPoint <urn:bldg/FCU2.fanLevel>,
        <urn:bldg/FCU2.mode>,
        <urn:bldg/FCU2.on>,
        <urn:bldg/FCU2.targetTemperature>,
        <urn:bldg/FCU2.temperature>,
        <urn:bldg/FCU2.time> ;
    ref:hasEx